# House Prices Prediction

## Importing:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# get_description() is a function to get the description of any column name or a value.
# It takes the column/value name, or any list of them:
from data_description import get_description

%matplotlib inline
sns.set_style('darkgrid')

### Importing the 'train.csv' and 'test.csv' files:

In [2]:
train = pd.read_csv('data/train.csv', index_col='Id')
test = pd.read_csv('data/test.csv', index_col='Id')

In [3]:
# X_train is the training features data
X_train = train.drop(['SalePrice'], axis=1)
# y is the target variable [SalePrice]
y = pd.DataFrame(train['SalePrice'])

# X is the combination of the features data from 'train' and 'test'. This is where we'll deal with the NaV values.
X = pd.concat([X_train, test], axis=0)

## Exploratory Data Analysis

### Deviding the features into categorical and numerical:

In [4]:
categorical = [x for x in X.columns if X[x].dtype == 'object']
numerical = [x for x in X.columns if X[x].dtype != 'object']

print(f'Numerical variables [{len(numerical)}]:\n{numerical}')
print()
print(f'Categorical variables [{len(categorical)}]:\n{categorical}')

Numerical variables [36]:
['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']

Categorical variables [43]:
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu',

### The features sliced into 5 sets:

In [5]:
feat_set1 = X.columns[0:16].tolist()
feat_set1

['MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle']

In [ ]:
feat_set2 = X.columns[16:32].tolist()
feat_set2

['OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure']

In [ ]:
feat_set3 = X.columns[32:48].tolist()
feat_set3

['BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath']

In [ ]:
feat_set4 = X.columns[48:64].tolist()
feat_set4

['FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond']

In [ ]:
feat_set4 = X.columns[64:80].tolist()
feat_set4

['PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SaleType',
 'SaleCondition']

### Analysis:

In [ ]:
feat_set1

['MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle']

In [ ]:
X['MSZoning'].value_counts()

RL         2265
RM          460
FV          139
RH           26
C (all)      25
Name: MSZoning, dtype: int64

In [ ]:
zoning = train.groupby('MSZoning')['SalePrice'].mean().round().sort_values().reset_index()
zoning.loc[0, 'MSZoning'] = 'C'
zoning['MSZoning'] = get_description(*zoning['MSZoning'])
fig = px.bar(zoning, y='MSZoning', x='SalePrice', color='SalePrice', orientation='h', text_auto=True)
fig.update_layout(title='Average Sale Price for general zoning classifications [MSZoning]')

In [ ]:

fig = px.scatter(train, x='LotArea', y='SalePrice', trendline='ols', trendline_color_override='red')
# fig.update_layout(xaxis_range=[0, 20000])
fig.update_layout(title='Average Sale Price for Lot Area [LotArea]')



In [ ]:
LandContour = train.groupby('LandContour')['SalePrice'].mean().round().sort_values().reset_index()
LandContour['LandContour'] = get_description(*LandContour['LandContour'])
fig = px.bar(LandContour, y='LandContour', x='SalePrice', color='SalePrice', orientation='h', text_auto=True)
fig.update_layout(title='Average Sale Price for Flatness of the property [LandContour]')

In [ ]:
var = 'LotConfig'
group = train.groupby(var)['SalePrice'].mean().round().sort_values().reset_index()
group[var] = get_description(*group[var])
fig = px.bar(group, y=var, x='SalePrice', color='SalePrice', orientation='h', text_auto=True)
fig.update_layout(title='Average Sale Price for Lot configurations')

In [ ]:
var = 'Condition1'
group = train.groupby(var)['SalePrice'].mean().round().sort_values().reset_index()
group[var] = get_description(*group[var])
fig = px.bar(group, y=var, x='SalePrice', color='SalePrice', orientation='h', text_auto=True)
fig.update_layout(title='Average Sale Price Vs Proximity to various conditions')

In [ ]:
var = 'BldgType'
group = train.groupby(var)['SalePrice'].mean().round().sort_values().reset_index()
group[var].replace({'Twnhs': 'TwnhsI', '2fmCon': '2FmCon'}, inplace=True)
group[var] = get_description(*group[var])
fig = px.bar(group, y=var, x='SalePrice', color='SalePrice', orientation='h', text_auto=True)
fig.update_layout(title='Average Sale Price for Building Type [BldgType]')

In [ ]:
var = 'HouseStyle'
group = train.groupby(var)['SalePrice'].mean().round().sort_values().reset_index()
group[var] = get_description(*group[var])
fig = px.bar(group, y=var, x='SalePrice', color='SalePrice', orientation='h', text_auto=True)
fig.update_layout(title='Average Sale Price Vs House Stype [HouseStyle]')

In [ ]:
X['Condition2'].value_counts(dropna=False)

Norm      2889
Feedr       13
Artery       5
PosN         4
PosA         4
RRNn         2
RRAn         1
RRAe         1
Name: Condition2, dtype: int64

## Preparing the Data:

#### Converting 'MSSubClass' to categorical:

In [ ]:
X['MSSubClass'] = X['MSSubClass'].astype('str')
X['MSSubClass']

Id
1        60
2        20
3        60
4        70
5        60
       ... 
2915    160
2916    160
2917     20
2918     85
2919     60
Name: MSSubClass, Length: 2919, dtype: object

### Missing Values:

In [ ]:
missing_count = X.isna().sum(axis=0)
missing_percent = missing_count / len(X) * 100

missing_data = pd.DataFrame({
    'NaN Count': missing_count,
    'Percentage [%]': missing_percent
}).sort_values(by='NaN Count', ascending=False)
missing_data.index.name = 'Column Name'

missing_data = missing_data[missing_data['NaN Count'] > 0]
missing_data

,NaN Count,Percentage [%]
Column Name,,
PoolQC,2909,99.657417
MiscFeature,2814,96.402878
Alley,2721,93.216855
Fence,2348,80.438506
FireplaceQu,1420,48.646797
LotFrontage,486,16.649538
GarageYrBlt,159,5.447071
GarageFinish,159,5.447071
GarageQual,159,5.447071


In [ ]:
# For numerical variables, replace with zero, when the house doesn't have the feature.
na_zero = []

# For numerical variables, replace with the mean, when the NaN means the value is missing.
na_mean = ['LotFrontage'] # median is better

# For categorical variables, replace with 'None', when the house doesn't have the feature.
na_none = ['Alley']

# For categorical variables, replace with the mode (most accuring value), where NaN means value is missing.
na_modes = ['MSZoning', 'Utilities']


X[feat_set1].isnull().sum()

MSSubClass         0
MSZoning           4
LotFrontage      486
LotArea            0
Street             0
Alley           2721
LotShape           0
LandContour        0
Utilities          2
LotConfig          0
LandSlope          0
Neighborhood       0
Condition1         0
Condition2         0
BldgType           0
HouseStyle         0
dtype: int64